In [5]:
import ee
import geemap
import collections
collections.Callable = collections.abc.Callable
ee.Initialize()
import pandas as pd

In [46]:
dicto = pd.read_csv('zipIncome.csv').to_dict()
data = ee.Dictionary(dicto)

In [52]:
Map = geemap.Map()

zips = ee.FeatureCollection("TIGER/2010/ZCTA5")
states = ee.FeatureCollection("TIGER/2016/States")

state = states.filter(ee.Filter.eq('NAME', 'California'))
ca = state.geometry()
zipsRange = zips.filter(ee.Filter.gte('GEOID10', '89010')) \
  .filter(ee.Filter.lte('GEOID10', '97635'))

def addCenter(feature):
    center = feature.centroid().geometry()
    insideCa = ca.contains(center, 1)
    return feature.set('containedIn', insideCa)

zipCenter = zipsRange.map(addCenter)

# filter the feature on metadata
zipInCa = zipCenter.filter(ee.Filter.eq("containedIn", True))
                          
def addIncome(feature):
    zippy = feature.get('GEOID10')
    if data.contains(zippy)==True:
        income = data.get(zippy)
        return feature.set('medianIncome', income)
    else:
        return feature.set('medianIncome', 0)

zipIncome = zipInCa.map(addIncome)

visParams = {'color': 'red'}
Map.addLayer(zipIncome, visParams, 'zip')
Map.centerObject(ca)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [40]:
first = zipIncome.first()
print(type(first.get('medianIncome').getInfo()))

<class 'dict'>
